# Assignment 1 : Yelp Dataset Analysis 

# Imports required for analysis

In [0]:
import numpy as np
#import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import seaborn as sns
import string
import pandas as pd 

# **Mounting data set from Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Importing Business.csv to get businness Id's that are Restaurant**

In [0]:
business=pd.read_csv('gdrive/My Drive/255-yelp-assignment1/yelp_business.csv',iterator=True, chunksize=1000)

df_business = pd.concat(business, ignore_index=True)

In [17]:
business_dataframe = df_business[df_business['categories'].str.contains('Restaurants','Restaurant', na=False)]
business_dataframe.shape

(54618, 13)

In [19]:
business_dataframe.head(5)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...
5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",NaN,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants
10,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""",NaN,"""737 West Pike St""",Houston,PA,15342,40.241548,-80.212815,4.5,3,0,Breakfast & Brunch;Gluten-Free;Coffee & Tea;Fo...
14,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional)
15,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""",Yonge and Eglinton,"""2459 Yonge St""",Toronto,ON,M4P 2H6,43.711399,-79.399339,3.0,12,0,Italian;French;Restaurants


**Removing unwanted column from business dataframe**

In [0]:
df_restaurant = business_dataframe.drop(columns=['neighborhood','address','city','postal_code','latitude','longitude','stars','review_count','is_open'])




#df_restaurant = df_business.loc[df_business['categories'].isin(['Restaurants','Restaurant','Food','Bars','Nightlife','Pubs','Salad','Food','Beer','Wine & Spirits','Restaurants','Mexican'])]

In [58]:
df_restaurant.head(5)

,business_id,name,state,categories
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...
5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",BW,Italian;Restaurants
10,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""",PA,Breakfast & Brunch;Gluten-Free;Coffee & Tea;Fo...
14,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",NC,Restaurants;American (Traditional)
15,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""",ON,Italian;French;Restaurants


In [61]:
df_restaurant.shape

(54618, 4)

**Loading Reviews from entire Review.csv dataset**

In [0]:
tp=pd.read_csv('gdrive/My Drive/255-yelp-assignment1/yelp_review.csv',iterator=True, chunksize=100000)

df_review = pd.concat(tp, ignore_index=True)

**Dropping unnecessary columns from the review data frame**

In [0]:
df_review = df_review.drop(columns=['review_id', 'user_id','date','useful','funny','cool'])

In [63]:
df_review.head()

,business_id,stars,text
0,AEx2SYEUJmTxVVB18LlCwA,5,Super simple place but amazing nonetheless. It...
1,VR6GpWIda3SfvPC-lg9H3w,5,Small unassuming place that changes their menu...
2,CKC0-MOWMqoeWf6s-szl8g,5,Lester's is located in a beautiful neighborhoo...
3,ACFtxLv8pGrrxMm6EgjreA,4,Love coming here. Yes the place always needs t...
4,s2I_Ni76bjJNK9yG60iD-Q,4,Had their chocolate almond croissant and it wa...


**Merging review data frame and restaurant data frame based on Business_ID that are restaurant**

In [0]:
df = pd.merge(df_restaurant, df_review, on = 'business_id') #Merging restaurants with review as per business id

In [65]:
df.head()

,business_id,name,state,categories,stars,text
0,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,4,I've always enjoyed my time at brick house foo...
1,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,2,1st time here. Came w my Unc bc Louies was clo...
2,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,1,Worse service ever andI use to be a server so ...
3,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,5,I am updating my review to 5-stars because I t...
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,4,I enjoyed this place. I went the night the Bu...


In [66]:
df.shape

(3221418, 6)

In [67]:
df.head()

,business_id,name,state,categories,stars,text
0,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,4,I've always enjoyed my time at brick house foo...
1,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,2,1st time here. Came w my Unc bc Louies was clo...
2,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,1,Worse service ever andI use to be a server so ...
3,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,5,I am updating my review to 5-stars because I t...
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,American (New);Nightlife;Bars;Sandwiches;Ameri...,4,I enjoyed this place. I went the night the Bu...


**Shuffling the data frame and picking up 20% of the data for further processing**

Reason to shuffle and select 20% data is due to limited processing and computing speed


In [0]:
#Shuffle DF to get random data set and select 20% of data out of 30,00,000 review. All this reviews are for business id that are restaurant
#Reason to shuffle and select 20% is because I am unable to process all the reviews giving the computing speed

from sklearn.utils import shuffle
df_shuffle = shuffle(df)
df_percent = df_shuffle.sample(frac=0.2)
df = df_percent

In [70]:
df.shape

(644284, 6)

**Data Preprocessing : Convert Review Text to lower case and removing punctuation**

In [0]:
## convert text to lower case
df.text = df.text.str.lower()
## remove unnecessary punctuation
df['removed_punct_text']= df.text.str.replace('\n',''). \
                                          str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


In [72]:
df.head(100)

,business_id,name,state,categories,stars,text,removed_punct_text
1053174,WCFz9KVJTtIXSVvD5NMYoQ,"""Chart House""",NV,Seafood;Event Planning & Services;American (Tr...,4,we made reservations to chart house on my birt...,we made reservations to chart house on my birt...
693097,-PKUAor_Nz_IaNjP_jIwyA,"""Playa Cabana - Hacienda""",ON,Restaurants;Mexican,3,"food was delicious- guac, the corn and the tac...",food was delicious guac the corn and the tacos...
1043630,2cib4FSxfh8OxRf9wX5RzQ,"""The Chocolate Moose""",NV,Desserts;Cafes;Ice Cream & Frozen Yogurt;Food;...,5,the most absolute awesome little eatery i've b...,the most absolute awesome little eatery i've b...
1840562,LgroNa-JH5W-wpJACXuz5A,"""Akita Sushi""",AZ,Restaurants;Sushi Bars,5,i can't praise akita sushi enough. the locatio...,i can't praise akita sushi enough the location...
2680949,klls1rEfABAUtO8J0YXSGQ,"""Aladdin's Eatery""",PA,Soup;Food;Mediterranean;Vegetarian;Juice Bars ...,4,this is one of the few chain restaurants i enj...,this is one of the few chain restaurants i enj...
2339070,1qwkbN3wu3AGDxMTXVz2gA,"""Topgolf""",AZ,Sports Bars;Bars;Active Life;Restaurants;Night...,4,we went to the location in gilbert. super fun ...,we went to the location in gilbert super fun a...
1824003,VqT-IURXmVS_fVRLTKxTFQ,"""Cacao 70""",QC,Chocolatiers & Shops;Breakfast & Brunch;Desser...,5,"i visited cacao 70 again tonight, and was agai...",i visited cacao 70 again tonight and was again...
2132370,l_GV0hgEoTUf70uJVT0_hg,"""Eat.""",NV,Restaurants;American (New);Sandwiches;Breakfas...,4,went here new year's day for breakfast with a ...,went here new year's day for breakfast with a ...
430283,Vs7gc9EE3k9wARuUcN9piA,"""Pan Asian""",NV,Japanese;Restaurants;Thai;Chinese,5,"awesome food, great spirit ,owners friendly ,s...",awesome food great spirit owners friendly sata...
3123035,JyxHvtj-syke7m9rbza7mA,"""Sushi House Goyemon""",NV,Sushi Bars;Japanese;Restaurants,5,sushi house goyemon is the best all-you-can-ea...,sushi house goyemon is the best allyoucaneat a...


**Creating List of Positive, Negative and Neutral word list**

In [73]:
positive_words=["good","great","love","best","fantastic","healthiest","definitely","authentic","professional","knowledgeable","confident","polite","friendly","free","wifi","pretty","priceless","privilege","prompt","awesome"]
negative_words=["horrible","slow","irritation","disappointed","awful","abruptly","ashamed","awful","worse"]
neutral_words=["fine","average","ok","satisfy","overall","unfortunately","much","wont","just","worth","still", "though"]
mergedlist = positive_words + negative_words+neutral_words
print(mergedlist)

['good', 'great', 'love', 'best', 'fantastic', 'healthiest', 'definitely', 'authentic', 'professional', 'knowledgeable', 'confident', 'polite', 'friendly', 'free', 'wifi', 'pretty', 'priceless', 'privilege', 'prompt', 'awesome', 'horrible', 'slow', 'irritation', 'disappointed', 'awful', 'abruptly', 'ashamed', 'awful', 'worse', 'fine', 'average', 'ok', 'satisfy', 'overall', 'unfortunately', 'much', 'wont', 'just', 'worth', 'still', 'though']


**Custom logic to label each review in the data set**

In [74]:
#reviews = "good yes"
import nltk
nltk.download('punkt')
Cust = []
from nltk.tokenize import sent_tokenize, word_tokenize
for r in df['text']: # counter for month indices
    review = word_tokenize(r)
    pos = 0
    neg = 0
    neu = 0
    for word in review:
        if word in mergedlist:
            if word in positive_words:
             pos = pos+1
            elif word in negative_words:
             neg = neg+1
            elif word in neutral_words:
             neu = neu +1
    #monthly_mean.append(s/n)
    if pos >= neg and pos >= neu:
     Cust.append('Positive')
    elif neg >= neu:
     Cust.append('Negative')
    else:
     Cust.append('Neutral') 
#print(pos)
#print(neg)
df['Predicted_review'] = Cust 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [75]:
df.head()

,business_id,name,state,categories,stars,text,removed_punct_text,Predicted_review
1053174,WCFz9KVJTtIXSVvD5NMYoQ,"""Chart House""",NV,Seafood;Event Planning & Services;American (Tr...,4,we made reservations to chart house on my birt...,we made reservations to chart house on my birt...,Positive
693097,-PKUAor_Nz_IaNjP_jIwyA,"""Playa Cabana - Hacienda""",ON,Restaurants;Mexican,3,"food was delicious- guac, the corn and the tac...",food was delicious guac the corn and the tacos...,Neutral
1043630,2cib4FSxfh8OxRf9wX5RzQ,"""The Chocolate Moose""",NV,Desserts;Cafes;Ice Cream & Frozen Yogurt;Food;...,5,the most absolute awesome little eatery i've b...,the most absolute awesome little eatery i've b...,Positive
1840562,LgroNa-JH5W-wpJACXuz5A,"""Akita Sushi""",AZ,Restaurants;Sushi Bars,5,i can't praise akita sushi enough. the locatio...,i can't praise akita sushi enough the location...,Positive
2680949,klls1rEfABAUtO8J0YXSGQ,"""Aladdin's Eatery""",PA,Soup;Food;Mediterranean;Vegetarian;Juice Bars ...,4,this is one of the few chain restaurants i enj...,this is one of the few chain restaurants i enj...,Neutral


In [76]:
df.shape

(644284, 8)

**Splitting the data for training and testing the model**

In [0]:
x = df['text']
y = df['Predicted_review']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 101)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv_transformer = CountVectorizer(analyzer = 'word')

In [0]:
x = cv_transformer.fit_transform(x)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 101)

# **Fitting the Classifier**

**1. Naive Bayes**

**2. Random Forest**

In [0]:
from sklearn.naive_bayes import MultinomialNB

# **Naive Bayes Classifier**

In [82]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [83]:
predictions = nb.predict(x_test)
predictions

array(['Positive', 'Positive', 'Positive', ..., 'Positive', 'Positive',
       'Positive'], dtype='<U8')

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

**Score for NB classifier**

In [85]:
print(confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

[[   826   2363   1665]
 [   117  17272  12553]
 [   764  20380 137346]]


              precision    recall  f1-score   support

    Negative       0.48      0.17      0.25      4854
     Neutral       0.43      0.58      0.49     29942
    Positive       0.91      0.87      0.89    158490

   micro avg       0.80      0.80      0.80    193286
   macro avg       0.61      0.54      0.54    193286
weighted avg       0.82      0.80      0.81    193286



# **2. Random Forest **

In [86]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(x_train,y_train)
predrmfr = rmfr.predict(x_test)
predrmfr

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array(['Positive', 'Positive', 'Positive', ..., 'Positive', 'Neutral',
       'Positive'], dtype=object)

**Scores for Random Forest**

In [87]:
print(confusion_matrix(y_test, predrmfr))
print("\n")
print(classification_report(y_test, predrmfr))

[[   203    261   4390]
 [    25   3851  26066]
 [    28   2642 155820]]


              precision    recall  f1-score   support

    Negative       0.79      0.04      0.08      4854
     Neutral       0.57      0.13      0.21     29942
    Positive       0.84      0.98      0.90    158490

   micro avg       0.83      0.83      0.83    193286
   macro avg       0.73      0.38      0.40    193286
weighted avg       0.79      0.83      0.78    193286



# **Analysis of data for NB classifier**

In [94]:
# Positive Misclassified review
pr = df['text'][1053174]
print(pr)
print("Actual Rating: ",df['stars'][1053174])
pr_t = cv_transformer.transform([pr])
print("Predicted Rating:", nb.predict(pr_t)[0])

we made reservations to chart house on my birthday since we were going to be downtown for the st. patrick's day festivities and i had a pretty decent group with me. having dinner in front of the fish tank was awesome but made me wonder if the fish i was eating was the lucky one to get plucked outta the tank. 

i was surprised that they had made a personalized menu for my table. apparently they ask if anyone is celebrating a special occasion when you call to make reservations. i thought this was pretty cool. plus they let me keep the personalized menus. a very nice gesture.

ok on to the food, the sea bass with saffron risotto was tasty. i think i enjoyed that better than my dish. i had the stuffed flounder, which was good but the flounder had no flavor and a little dry until you mixed it with the crab. the sea bass though, mmm. i also heard that the jumbo stuffed shrimp, surf & turf, and fish & chips were very enjoyable. 

overall the experience was awesome especially since it was in d

In [99]:
# Neutral Classified Review 
pr = df['text'][693097]
print(pr)
print("Actual Rating: ",df['stars'][693097])
pr_t = cv_transformer.transform([pr])
print("Predicted Rating:", nb.predict(pr_t)[0])

food was delicious- guac, the corn and the tacos... delicious! the service was nothing over the moon. someone takes your order, 20 other people drop it off and the server no where to be seen. we wanted to get desert and another drink, but the waiter said that they had another rsvp for our 2 seater table in 15mins, even though there were 6 available tables surrounding us upstairs.
Actual Rating:  3
Predicted Rating: Neutral


In [101]:
# Positive Misclassified Review 
pr = df['text'][1732208]
print(pr)
print("Actual Rating: ",df['stars'][1732208])
pr_t = cv_transformer.transform([pr])
print("Predicted Rating:",nb.predict(pr_t)[0])

no seriously. this is not sushi, this is some sort of montreal fusion sushi-lookalike.
Actual Rating:  1
Predicted Rating: Positive


In [102]:
pr = df['text'][2462289]
print(pr)
print("Actual Rating: ",df['stars'][2462289])
pr_t = cv_transformer.transform([pr])
print("Predicted Rating:",nb.predict(pr_t)[0])

okay service. waited long for one worker to serve 3 tables. didn't get my complimentary bread and everyone around who is local got it. food was room temperature, not hot. if you are american and don't speak french, don't go here.
Actual Rating:  1
Predicted Rating: Positive


# **Random Forest Classifier - Analysis of result**

In [104]:
# Positive Classified review
pr = df['text'][1053174]
print(pr)
print("Actual Rating: ",df['stars'][1053174])
pr_t = cv_transformer.transform([pr])
print("Predicted Rating:", rmfr.predict(pr_t)[0])

we made reservations to chart house on my birthday since we were going to be downtown for the st. patrick's day festivities and i had a pretty decent group with me. having dinner in front of the fish tank was awesome but made me wonder if the fish i was eating was the lucky one to get plucked outta the tank. 

i was surprised that they had made a personalized menu for my table. apparently they ask if anyone is celebrating a special occasion when you call to make reservations. i thought this was pretty cool. plus they let me keep the personalized menus. a very nice gesture.

ok on to the food, the sea bass with saffron risotto was tasty. i think i enjoyed that better than my dish. i had the stuffed flounder, which was good but the flounder had no flavor and a little dry until you mixed it with the crab. the sea bass though, mmm. i also heard that the jumbo stuffed shrimp, surf & turf, and fish & chips were very enjoyable. 

overall the experience was awesome especially since it was in d

In [105]:
# Neutral Classified Review 
pr = df['text'][693097]
print(pr)
print("Actual Rating: ",df['stars'][693097])
pr_t = cv_transformer.transform([pr])
print("Predicted Rating:", rmfr.predict(pr_t)[0])

food was delicious- guac, the corn and the tacos... delicious! the service was nothing over the moon. someone takes your order, 20 other people drop it off and the server no where to be seen. we wanted to get desert and another drink, but the waiter said that they had another rsvp for our 2 seater table in 15mins, even though there were 6 available tables surrounding us upstairs.
Actual Rating:  3
Predicted Rating: Neutral


**Conclusion:**
The data set being huge I could only process 3Lakh reviews succesfully.

**Used 4 classifier algorithms namely :**

Naive Bayes
Random Forest
Knn
SVM

For data size more than 50000 and data less than 300000, knn and SVM could fit the data. When data increased to above 3Lakh it failed to fit the data.

Naive Bayes and Random Forest worked prettly good in all scenarios, Random forest being the best out of all algorithms.

